In [ ]:
# Import packages needed
import numpy as np 
import pandas as pd
from scipy import stats 
from statsmodels.stats.proportion import proportions_ztest
from statsmodels.stats.proportion import binom_test
from scipy.stats import binom_test

/usr/local/lib/python3.7/dist-packages/statsmodels/tools/_testing.py:19: FutureWarning: pandas.util.testing is deprecated. Use the functions in the public API at pandas.testing instead.
  import pandas.util.testing as tm


## Esitmate Standard Deviation of Evaluate Metrics

We need to calculate the standard deviation for each evaluation metric. 







Baseline Value, provided by Udacity

| Metric | Description |Baseline Value|Sampled Value|
| --- | ----------- |----------- |----------- |
| Number of Cookies | Unique cookies to view course overview page per day | 40000| 5000|
| Number of Clicks | Unique cookies to click "Start free trial" per day | 3200 | |
| Number of Enrollments | Enrollments per day | 660|
| CTP | Click-through-probability on "Start free trial"|0.08| |
| Gross Conversion | Probability of enrolling, given click|0.20625| |
| Retention | Probability of payment, given enroll |0.53| |
| Net Conversion | Probability of payment, given click |0.1093125| |


In [ ]:
# created a dataframe to store the data listed above
d = {'Metric': ['Number of Cookies', 'Number of Clicks','Number of Enrollments','CTP','Gross Conversion','Retention','Net Conversion'], 
     'Baseline_val': [40000, 3200, 660, 0.08, 0.20625,0.53, 0.1093125]
     }
me = pd.DataFrame(data=d)
me

,Metric,Baseline_val
0,Number of Cookies,40000.000000
1,Number of Clicks,3200.000000
2,Number of Enrollments,660.000000
3,CTP,0.080000
4,Gross Conversion,0.206250
5,Retention,0.530000
6,Net Conversion,0.109313


Since the sample size given by Udacity is n = 5000 cookies, we first need to scale the number of cookies, the number of clicks and the number of user-ids as well.

In [ ]:
#create new column to store sampled value
me.insert (2, "Sampled_val", np.nan)
me

,Metric,Baseline_val,Sampled_val
0,Number of Cookies,40000.000000,NaN
1,Number of Clicks,3200.000000,NaN
2,Number of Enrollments,660.000000,NaN
3,CTP,0.080000,NaN
4,Gross Conversion,0.206250,NaN
5,Retention,0.530000,NaN
6,Net Conversion,0.109313,NaN


In [ ]:
coef = 5000/40000
coef

0.125

In [ ]:
# Calculate Number of clicks, number of enrollments based on sampled number of cookies
for i in range(3):
  me.at[i,'Sampled_val'] = me.loc[i]["Baseline_val"] * coef

me

,Metric,Baseline_val,Sampled_val
0,Number of Cookies,40000.000000,5000.0
1,Number of Clicks,3200.000000,400.0
2,Number of Enrollments,660.000000,82.5
3,CTP,0.080000,NaN
4,Gross Conversion,0.206250,NaN
5,Retention,0.530000,NaN
6,Net Conversion,0.109313,NaN


Since each evaluation metric is probability and we can assume the metrics are binominal distributed, so we can calculate the standard deviation with the formula below:



*   <font size=3> $ SD = \sqrt{p*(1-p)\over N}$</font>


*   p - baseline probability of the event to occur
*   N - sample size

In [ ]:
# Add a column to store the standard deviation of evaluate metrics.
me.insert(3,'SD',np.nan)
me

,Metric,Baseline_val,Sampled_val,SD
0,Number of Cookies,40000.000000,5000.0,NaN
1,Number of Clicks,3200.000000,400.0,NaN
2,Number of Enrollments,660.000000,82.5,NaN
3,CTP,0.080000,NaN,NaN
4,Gross Conversion,0.206250,NaN,NaN
5,Retention,0.530000,NaN,NaN
6,Net Conversion,0.109313,NaN,NaN


In [ ]:
# Calculate the standard error of Gross Convension
me.at[4,'SD'] = np.sqrt(me.loc[4]['Baseline_val']*(1-me.loc[4]['Baseline_val'])/me.loc[1]['Sampled_val'])

# Calculate the standard error of Retention
me.at[5,'SD'] = np.sqrt(me.loc[5]['Baseline_val']*(1-me.loc[5]['Baseline_val'])/me.loc[2]['Sampled_val'])

# Calculate the standard error of Net Convension
me.at[6,'SD'] = np.sqrt(me.loc[6]['Baseline_val']*(1-me.loc[6]['Baseline_val'])/me.loc[1]['Sampled_val'])

me

,Metric,Baseline_val,Sampled_val,SD
0,Number of Cookies,40000.000000,5000.0,NaN
1,Number of Clicks,3200.000000,400.0,NaN
2,Number of Enrollments,660.000000,82.5,NaN
3,CTP,0.080000,NaN,NaN
4,Gross Conversion,0.206250,NaN,0.020231
5,Retention,0.530000,NaN,0.054949
6,Net Conversion,0.109313,NaN,0.015602


## Experiment Size

After estimating our metrics in the baseline and their estimated variance, we can move forward to calculate the minimal sample size so that the experiment will have enough statistical power and also statistical siginificance. 

In this experiment, α=0.05  (significance level ) and  1-β=0.8  (power). The sample size could be calculated by following fromula:


  <font size=4> $n = \frac{(Z_ {1-\frac{\alpha}{2}}  sd_1 + Z_ {1-\beta} sd_2)^2}{d^2} $  </font>

  <font size=3> $sd_1 = \sqrt{2*p*(1-p)}$</font>

 <font size=3> $ sd_2 = \sqrt{p*(1-p)+(p+d_{min})(1-(p+d_{min}))  }$</font>


When we calculate the experiment sample size we have to keep in mind that n indicates the sample size per group and we have two groups here, experiment group and control group. Further, the experiment sample size here is the number of cookies that visit the page, so we also need to care about the circumstance that the evaluation metrics' units of analysis are number of clicks and number of enrollments, respectively.


In [ ]:
# Add minimal detectable effect value (provided by Udacity) into me dataframe.
dmin = [3000, 50, 240, 0.01, 0.01, 0.01, 0.0075]
me['dmin'] = dmin

# Add a column to store the sample size
me.insert(5, 'size', np.nan)
me

,Metric,Baseline_val,Sampled_val,SD,dmin,size
0,Number of Cookies,40000.000000,5000.0,NaN,3000.0000,NaN
1,Number of Clicks,3200.000000,400.0,NaN,50.0000,NaN
2,Number of Enrollments,660.000000,82.5,NaN,240.0000,NaN
3,CTP,0.080000,NaN,NaN,0.0100,NaN
4,Gross Conversion,0.206250,NaN,0.020231,0.0100,NaN
5,Retention,0.530000,NaN,0.054949,0.0100,NaN
6,Net Conversion,0.109313,NaN,0.015602,0.0075,NaN


In [ ]:
def get_sd(p,d):
  '''
  p - baseline value
  d - dmin

  '''
  sd_1 = np.sqrt(2*p*(1-p))
  sd_2 = np.sqrt(p*(1-p)+(p+d)*(1-p-d))
  sd = [sd_1, sd_2]
  return sd

In [ ]:
# calculate sample size n by the formula listed above
def sample_size(sd,alpha,beta,dmin):
  '''
  sd - list, results from get_sd function
  alpha - significance level, usually 0.05
  beta - (1-beta) is statistical power
  dmin - minimum detectable effect
  '''
  n = pow((stats.norm.ppf(1-alpha/2)*sd[0] + stats.norm.ppf(1-beta)*sd[1]),2)/pow(dmin,2)
  return n 

### Calculate sample size per metric

#### Gross Conversion

In [ ]:
# Gross Conversion
temp = sample_size(get_sd(me.loc[4]['Baseline_val'],me.loc[4]['dmin']),0.05,0.2,me.loc[4]['dmin'])
print('We need %s clicks per group'%(np.ceil(temp)))

We need 25835.0 clicks per group


In [ ]:
# Unit of analysis for Gross Conversion is number of click, but what we need is number of cookies.
n = np.ceil(temp*2/(3200/40000))
n

645868.0

In [ ]:
me.at[4,'size'] = n

#### Retention

In [ ]:
# Retention
temp = sample_size(get_sd(me.loc[5]['Baseline_val'],me.loc[5]['dmin']),0.05,0.2,me.loc[5]['dmin'])
print('We need %s enrollments per group'%(np.ceil(temp)))

We need 39087.0 enrollments per group


In [ ]:
# Unit of analysis for Retention is number of enrollments, but what we need is number of cookies.
n = np.ceil(temp*2/(660/40000))
me.at[5,'size'] = n
n

4737771.0

#### Net Conversion

In [ ]:
# Net Conversion
temp = sample_size(get_sd(me.loc[6]['Baseline_val'],me.loc[6]['dmin']),0.05,0.2,me.loc[6]['dmin'])
print('We need %s clicks per group'%(np.ceil(temp)))

We need 27414.0 clicks per group


In [ ]:
# Unit of analysis for Net Conversion is number of click, but what we need is number of cookies.
n = np.ceil(temp*2/(3200/40000))
me.at[6,'size'] = n
n

685334.0

In [ ]:
me

,Metric,Baseline_val,Sampled_val,SD,dmin,size
0,Number of Cookies,40000.000000,5000.0,NaN,3000.0000,NaN
1,Number of Clicks,3200.000000,400.0,NaN,50.0000,NaN
2,Number of Enrollments,660.000000,82.5,NaN,240.0000,NaN
3,CTP,0.080000,NaN,NaN,0.0100,NaN
4,Gross Conversion,0.206250,NaN,0.020231,0.0100,645868.0
5,Retention,0.530000,NaN,0.054949,0.0100,4737771.0
6,Net Conversion,0.109313,NaN,0.015602,0.0075,685334.0


## Experiment Analysis

In [ ]:
# Loading results data

# load data collected from control group as con
con = pd.read_excel('/content/drive/MyDrive/Results Data.xlsx',sheet_name='Control'); 

# load data collected from experiment group as exp
exp = pd.read_excel('/content/drive/MyDrive/Results Data.xlsx',sheet_name='Experiment');

In [ ]:
exp.head()

,Date,Pageviews,Clicks,Enrollments,Payments
0,"Sat, Oct 11",7716,686,105.0,34.0
1,"Sun, Oct 12",9288,785,116.0,91.0
2,"Mon, Oct 13",10480,884,145.0,79.0
3,"Tue, Oct 14",9867,827,138.0,92.0
4,"Wed, Oct 15",9793,832,140.0,94.0


In [ ]:
exp.count()

Date           37
Pageviews      37
Clicks         37
Enrollments    23
Payments       23
dtype: int64

In [ ]:
con.head()

,Date,Pageviews,Clicks,Enrollments,Payments
0,"Sat, Oct 11",7723,687,134.0,70.0
1,"Sun, Oct 12",9102,779,147.0,70.0
2,"Mon, Oct 13",10511,909,167.0,95.0
3,"Tue, Oct 14",9871,836,156.0,105.0
4,"Wed, Oct 15",10014,837,163.0,64.0


In [ ]:
con.count()

Date           37
Pageviews      37
Clicks         37
Enrollments    23
Payments       23
dtype: int64

### Sanity Check

Before we really perform complicated analysis on results, we need to do the sanity checks to verify the experiment conducted as expected and the data collected correctly.

We have 3 Invariant Metrics:

1.   Number of Cookies in Course Overview Page
2.   Number of Clicks on Free Trial Button
3.   Free Trial button Click-Through-Probability

And we expect to see that these metrics not to vary between control group and experiment group, to be more precise, we need to verify there's no significant difference between metrics in two groups.



In [ ]:
#create a dataframe to store sanity check results
# Exp - expected value
# Obs - observed value
# CI_Left - left bound of Confidence Interval
# CI_Right - Right bound of Confidence Interval

sanity_check = pd.DataFrame(columns=["Exp", "Obs", "CI_left", "CI_right", "Pass"], index=["Cookies", "Clicks", "CTP"])

#### Number of Cookies & Number of Clicks

Cookies are randomly assigned to experiment group or control group with a porbability of 0.5, so it's binomial distributed.



1.   Calculated standard deviation of binomial with 0.5 probability of success.

  <font size=3> $SD = \sqrt{p*(1-p)\over N}$</font>
2.   Multiply Z-score to get a confidence interval around 0.5 (with 0.95 confidence level, Z-score = 1.96)

3.  Check if the observed value falls into the confidence interval





In [ ]:
for i in ('Cookies','Clicks'):
  sanity_check.at[i,'Exp'] = 0.5
sanity_check

,Exp,Obs,CI_left,CI_right,Pass
Cookies,0.5,NaN,NaN,NaN,NaN
Clicks,0.5,NaN,NaN,NaN,NaN
CTP,NaN,NaN,NaN,NaN,NaN


In [ ]:
def SanityCheck(metric:str):
  '''
  Pageviews or Clicks

  '''
  exp_val = exp[metric].sum()
  con_val = con[metric].sum()
  SD = np.sqrt(0.5*0.5/(exp_val+con_val)) # compute standard deviation
  left = round(0.5 - SD*1.96,4) # left bound of confidence interval
  right = round(0.5 + SD*1.96,4) # right bound of confidence interval
  obs = round(exp_val/(exp_val+con_val),4) # observed value 
  if_pass = ''
  if left <= obs <= right:  # check if the observed value in confidence interval
    if_pass = 'Yes'
  else:
    if_pass = 'No'
  res = [obs, left, right, if_pass]
  return res



In [ ]:
sanity_check.at['Cookies',["Obs", "CI_left", "CI_right", "Pass"]] = SanityCheck('Pageviews')
sanity_check.at['Clicks',["Obs", "CI_left", "CI_right", "Pass"]] = SanityCheck('Clicks')
sanity_check

,Exp,Obs,CI_left,CI_right,Pass
Cookies,0.5,0.4994,0.4988,0.5012,Yes
Clicks,0.5,0.4995,0.4959,0.5041,Yes
CTP,0,0.0001,-0.0013,0.0013,Yes


#### CTP

To do the sanity check for the click-through probabilites, we conduct a two proportion z-test with a click being interpreted as a success. We thereby assume that the two populations are normal distributed (may have difference variance). To perform the test, we can calculate a confidence interval around the expected difference of the two metrics, 0 in this case.

<font size=3> $Confidence Interval = [0-Z_ {1-\frac{\alpha}{2}}*SE_{pooled},0 + Z_ {1-\frac{\alpha}{2}}*SE_{pooled}] $

$\alpha = 0.05$  
</font>

<font size=3> $SE_{pooled} = \sqrt{{\frac{{S_{exp}}^{2}}{N_{exp}}+ {\frac{{S_{con}}^{2}}{N_{con}}}}} $  
</font>

<font size=3> $S^2 = p*(1-p)$, $S^2$ is variance of the sample 
</font>

$P$ is CTP here.





In [ ]:
exp.head()

,Date,Pageviews,Clicks,Enrollments,Payments
0,"Sat, Oct 11",7716,686,105.0,34.0
1,"Sun, Oct 12",9288,785,116.0,91.0
2,"Mon, Oct 13",10480,884,145.0,79.0
3,"Tue, Oct 14",9867,827,138.0,92.0
4,"Wed, Oct 15",9793,832,140.0,94.0


In [ ]:
CTP_exp = exp.Clicks.sum()/exp.Pageviews.sum();
CTP_con = con.Clicks.sum()/con.Pageviews.sum();
print('CTP for control group is %s, \nCTP for experiment group is %s.'%(CTP_con,CTP_exp))

CTP for control group is 0.08212581357457682, 
CTP for experiment group is 0.08218244066616376.


In [ ]:
cookie_con = con.Pageviews.sum();
cookie_exp = exp.Pageviews.sum();

# Compute sample variance for both groups
var_con = CTP_con*(1-CTP_con);
var_exp = CTP_exp*(1-CTP_exp);
SE_pool = np.sqrt(var_exp/cookie_exp + var_con/cookie_con)
SE_pool

0.0006610610775037591

In [ ]:
CTP_diff = round(CTP_exp-CTP_con, 4)
CTP_diff

0.0001

In [ ]:
# Z-score for 95% confidence level is about 1.96
sanity_check.at['CTP',["Exp","Obs"]] = [0, CTP_diff]

# Calculate left bound of Confidence Interval             
sanity_check.at['CTP',"CI_left"] = 0 - round(1.96*SE_pool,4);

# Calculate right bound of Confidence Interval             
sanity_check.at['CTP',"CI_right"] = 0 + round(1.96*SE_pool,4);

# Check if the observed value is within the confidence interval.
if sanity_check.loc['CTP']['CI_left'] < sanity_check.loc['CTP']['Obs'] < sanity_check.loc['CTP']['CI_right']:
  sanity_check.at['CTP','Pass'] = 'Yes'
else:
  sanity_check.at['CTP','Pass'] = 'No'

sanity_check

,Exp,Obs,CI_left,CI_right,Pass
Cookies,0.5,0.4994,0.4988,0.5012,Yes
Clicks,0.5,0.4995,0.4959,0.5041,Yes
CTP,0,0.0001,-0.0013,0.0013,Yes


 Another approach for two proportion Z-test is calculating the Z-test-statistic and then checking the corresponding p-value.
 
  <font size=4> $Z = \frac{CTP_{exp}-CTP_{con}}{\sqrt{{\frac{{S_{exp}}^{2}}{N_{exp}}+ {\frac{{S_{con}}^{2}}{N_{con}}}}}}$</font>

### Effect Size Test

Similar to the click-through probability, we can test our evaluation metric hypotheses using two proportion z-test (thereby, the same assumptions as outlined above apply). Further, we will check if the changes are also practically significant (if larger than dmin)

Recall our hypotheses:

$H_0: Gross Conversion_{exp}= Gross Conversion_{con}$   
$H_1: Gross Conversion_{exp}≠ Gross Conversion_{con}$   
$H_0: Net Conversion_{exp}= Net Conversion_{con}$   
$H_1: Net Conversion_{exp}≠ Net Conversion_{con}$   

Note: As we see in the test data, "payments" and "enrollments" were only tracked for 23 days. 


In [ ]:
# Make a copy of data collected from 23 days 
exp_1 = exp.dropna()
con_1 = con.dropna()

In [ ]:
#create a dataframe to store test results
# diff - difference between metrics in two groups
# CI_Left - left bound of Confidence Interval
# CI_Right - Right bound of Confidence Interval
# dmin - minimum detectable effect, given by Udacity

test = pd.DataFrame(columns=["diff", "CI_left", "CI_right", "dmin","stat. significant","Prac. significant"],
                    index=["Gross_Conv", "Net_Conv"])
test.dmin = [0.01,0.0075]
test

,diff,CI_left,CI_right,dmin,stat. significant,Prac. significant
Gross_Conv,NaN,NaN,NaN,0.0100,NaN,NaN
Net_Conv,NaN,NaN,NaN,0.0075,NaN,NaN


In [ ]:
def get_result(m,n,d:str):
 '''
 m - metric, Gross_Conv or Net_Conv.
 n - numerator of the metric
 d - denominator of the metric (unit of analysis)

 '''
 # Compute the evaluation metric for two groups
 metric_exp = exp_1[n].sum()/exp_1[d].sum()
 metric_con = con_1[n].sum()/con_1[d].sum()

 # Compute the observed difference between experiment and control metric
 diff = metric_exp - metric_con

 # Compute the Variance of the metric, var = p*(1-p)
 var_exp = metric_exp*(1-metric_exp)
 var_con = metric_con*(1-metric_con)
 
 # Compute the pooled SE
 SE = np.sqrt(var_exp/exp_1[d].sum() + var_con/con_1[d].sum())

 # Compute the left and right bound of confidence interval, with alpha = 0.05 (Z-score = 1.96)
 left = round(diff - 1.96*SE,4)
 right = round(diff + 1.96*SE,4)
 
 # Check if statistically significant, (if 0 within confidence interval)
 stats_sig = ''
 if left <= 0 <= right:
   stats_sig = 'No'
 else:
   stats_sig = 'Yes'
 
 # Check if practically significant
 prac_sig = ''
 if abs(diff) > test.loc[m]['dmin']:
   prac_sig = 'Yes'
 else:
   prac_sig = 'No'
  
 return [round(diff,4), left, right, stats_sig, prac_sig]


In [ ]:
test.at['Gross_Conv',["diff", "CI_left", "CI_right", "stat. significant","Prac. significant"]] = get_result('Gross_Conv', 'Enrollments','Clicks')

In [ ]:
test.at['Net_Conv',["diff", "CI_left", "CI_right", "stat. significant","Prac. significant"]] = get_result('Gross_Conv', 'Payments','Clicks')

In [ ]:
test

,diff,CI_left,CI_right,dmin,stat. significant,Prac. significant
Gross_Conv,-0.0206,-0.0291,-0.012,0.0100,Yes,Yes
Net_Conv,-0.0049,-0.0116,0.0019,0.0075,No,No


#### Double check with Sign Tests 

In a sign test we get another perspective at analyzing the results we got - we check if the trend of change we observed (increase or decrease) was evident in the daily data. We are going to compute the metric's value per day, then count on how many days the metric was lower in the experiment group and this will be the number of successes for our binomial variable. Once this is defined we can look at the proportion of days of success out of all the available days.

In [ ]:
# Prepare the data
full=con_1.join(other=exp_1,how="inner",lsuffix="_con",rsuffix="_exp");
full.head()

,Date_con,Pageviews_con,Clicks_con,Enrollments_con,Payments_con,Date_exp,Pageviews_exp,Clicks_exp,Enrollments_exp,Payments_exp
0,"Sat, Oct 11",7723,687,134.0,70.0,"Sat, Oct 11",7716,686,105.0,34.0
1,"Sun, Oct 12",9102,779,147.0,70.0,"Sun, Oct 12",9288,785,116.0,91.0
2,"Mon, Oct 13",10511,909,167.0,95.0,"Mon, Oct 13",10480,884,145.0,79.0
3,"Tue, Oct 14",9871,836,156.0,105.0,"Tue, Oct 14",9867,827,138.0,92.0
4,"Wed, Oct 15",10014,837,163.0,64.0,"Wed, Oct 15",9793,832,140.0,94.0


In [ ]:
# Compute daily Gross Conversion for both groups
full['Gross_Conv_con'] = full['Enrollments_con']/full['Clicks_con']
full['Gross_Conv_exp'] = full['Enrollments_exp']/full['Clicks_exp']

# Compute daily Net Conversion for both groups
full['Net_Conv_con'] = full['Payments_con']/full['Clicks_con']
full['Net_Conv_exp'] = full['Payments_exp']/full['Clicks_exp']

full.drop(columns = 'Date_exp',axis=1,inplace=True)
full.head()

,Date_con,Pageviews_con,Clicks_con,Enrollments_con,Payments_con,Pageviews_exp,Clicks_exp,Enrollments_exp,Payments_exp,Gross_Conv_con,Gross_Conv_exp,Net_Conv_con,Net_Conv_exp
0,"Sat, Oct 11",7723,687,134.0,70.0,7716,686,105.0,34.0,0.195051,0.153061,0.101892,0.049563
1,"Sun, Oct 12",9102,779,147.0,70.0,9288,785,116.0,91.0,0.188703,0.147771,0.089859,0.115924
2,"Mon, Oct 13",10511,909,167.0,95.0,10480,884,145.0,79.0,0.183718,0.164027,0.104510,0.089367
3,"Tue, Oct 14",9871,836,156.0,105.0,9867,827,138.0,92.0,0.186603,0.166868,0.125598,0.111245
4,"Wed, Oct 15",10014,837,163.0,64.0,9793,832,140.0,94.0,0.194743,0.168269,0.076464,0.112981


In [ ]:
# Compute number of success cases, as we define that the metric being lower in the experiment group considered as SUCCESS.
full['Gross_Conv_sign'] = np.where(full['Gross_Conv_exp'] < full['Gross_Conv_con'],1,0)
full['Net_Conv_sign'] = np.where(full['Net_Conv_exp'] < full['Net_Conv_con'],1,0)
full.head(10)

,Date_con,Pageviews_con,Clicks_con,Enrollments_con,Payments_con,Pageviews_exp,Clicks_exp,Enrollments_exp,Payments_exp,Gross_Conv_con,Gross_Conv_exp,Net_Conv_con,Net_Conv_exp,Gross_Conv_sign,Net_Conv_sign
0,"Sat, Oct 11",7723,687,134.0,70.0,7716,686,105.0,34.0,0.195051,0.153061,0.101892,0.049563,1,1
1,"Sun, Oct 12",9102,779,147.0,70.0,9288,785,116.0,91.0,0.188703,0.147771,0.089859,0.115924,1,0
2,"Mon, Oct 13",10511,909,167.0,95.0,10480,884,145.0,79.0,0.183718,0.164027,0.104510,0.089367,1,1
3,"Tue, Oct 14",9871,836,156.0,105.0,9867,827,138.0,92.0,0.186603,0.166868,0.125598,0.111245,1,1
4,"Wed, Oct 15",10014,837,163.0,64.0,9793,832,140.0,94.0,0.194743,0.168269,0.076464,0.112981,1,0
5,"Thu, Oct 16",9670,823,138.0,82.0,9500,788,129.0,61.0,0.167679,0.163706,0.099635,0.077411,1,1
6,"Fri, Oct 17",9008,748,146.0,76.0,9088,780,127.0,44.0,0.195187,0.162821,0.101604,0.056410,1,1
7,"Sat, Oct 18",7434,632,110.0,70.0,7664,652,94.0,62.0,0.174051,0.144172,0.110759,0.095092,1,1
8,"Sun, Oct 19",8459,691,131.0,60.0,8434,697,120.0,77.0,0.189580,0.172166,0.086831,0.110473,1,0
9,"Mon, Oct 20",10667,861,165.0,97.0,10496,860,153.0,98.0,0.191638,0.177907,0.112660,0.113953,1,0


In [ ]:
# Computing P value
for m in ('Gross_Conv','Net_Conv'):
  i = m + '_sign'
  p_val = binom_test(x= full[i].sum(), n = full[i].count(), p=0.5)

  # If p value is smaller than significance level 0.05
  if p_val < 0.05:
    print('P_val for %s is %s,  %s is statistically significant.'%(i, p_val, m))
  else:
    print('P_val for %s is %s,  %s is NOT statistically significant.'%(i, p_val, m))

P_val for Gross_Conv_sign is 0.0025994777679443364,  Gross_Conv is statistically significant.
P_val for Net_Conv_sign is 0.6776394844055175,  Net_Conv is NOT statistically significant.


Sign test results match the effect size calculation.